In [9]:
import datetime
import getpass
import json
import urllib

import sqlalchemy as sa

from sqlalchemy.engine import create_engine
from sqlalchemy.orm import declarative_base, Session

In [2]:
your_password_here = getpass.getpass()
conn_str = (
    'Driver={ODBC Driver 17 for SQL Server};'
    'Server=tcp:paper-scope-dbserver.database.windows.net,1433;'
    'Database=paper-scope-db;'
    'Uid=brian;'
    f'Pwd={your_password_here};'
    'Encrypt=yes;'
    'TrustServerCertificate=no;'
    'Connection Timeout=60;'
)

In [30]:
params = urllib.parse.quote_plus(conn_str)
engine = create_engine(f'mssql+pyodbc:///?odbc_connect={params}', echo=False)

In [42]:
Base = declarative_base()

In [43]:
class Article(Base):
    __tablename__ = 'article'

    id = sa.Column(sa.Integer, primary_key=True)
    href = sa.Column(sa.String(256))
    title = sa.Column(sa.String(256))
    description = sa.Column(sa.String(1024))
    image = sa.Column(sa.String(512))
    tags = sa.Column(sa.String(512))
    groups = sa.Column(sa.String(512))
    author = sa.Column(sa.String(64))
    date = sa.Column(sa.DateTime)
    content = sa.Column(sa.String(None))

    def __repr__(self):
        return (
            f'Article(id={self.id!r}, href={self.href!r}, title={self.title!r}'
            f', description={self.description[:30]!r}, image={self.image!r}'
            f', tags={self.tags!r}, groups={self.groups!r}, author={self.author!r}'
            f', date={self.date!r}, content={self.content[:30]!r}'
        )

In [44]:
Base.metadata.drop_all(engine)
Base.metadata.create_all(engine)

In [45]:
with Session(engine) as session:
    with session.begin():
        for article in json.load(open('../current.json')):
            article['groups'] = json.dumps(article['groups'])
            article['date'] = datetime.datetime.fromisoformat(article['date'])
            session.add(Article(**article))